In [2]:
import matplotlib.pyplot as plt

import matplotlib.patches as patches
import numpy as np
import group_glm as gm
from glm_utils import *
from scipy.stats import linregress
import pprint
import allensdk.brain_observatory.stimulus_info as stim_info
from allensdk.core.brain_observatory_cache import BrainObservatoryCache
import tensorflow as tf
import warnings
import pandas as pd

warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
%matplotlib inline

boc = BrainObservatoryCache(manifest_file='boc/manifest.json')

In [3]:
#first we get the data

regions = ['VISpm']
lines =  ['Cux2-CreERT2']#, 'Rbp4-Cre', 'Rorb-IRES2-Cre'] 

data_set = download_data(regions, lines, [stim_info.NATURAL_SCENES])

In [4]:
all_tensors = {}
ra_variance = {}
dffs = {}
data_dict = {}

var_thresh = .1
for key in data_set.keys():
    dff, images, stim_table = get_data(data_set[key], stim_info.NATURAL_SCENES)
    data_tensor, trialCount = arrange_ns_data_trialTensor(dff, stim_table)
    all_tensors[key] = data_tensor
    
    ra_variance[key] = get_explainable_variance(all_tensors[key])
    
    exp_var = ra_variance[key]
    exp_ind = exp_var > var_thresh
    
    pruned_dff_traces = dff[exp_ind]
    
    print "neurons with >", 100 * var_thresh, "% explainable variance for dataset: ", key, " : ", sum(exp_ind)
    
    r_images = pca_features(images)
    
    data, stim_array = arrange_data_glm(pruned_dff_traces, r_images, stim_table)
    
    dxcm, dtime = data_set[key].get_running_speed()
    rs = arrange_rs_glm(dxcm, stim_table)
    
    data_dict[key] = (data, stim_array, rs)
    

neurons with > 10.0 % explainable variance for dataset:  503866276  :  32
neurons with > 10.0 % explainable variance for dataset:  512145745  :  11
neurons with > 10.0 % explainable variance for dataset:  504508104  :  11
neurons with > 10.0 % explainable variance for dataset:  509517577  :  4
neurons with > 10.0 % explainable variance for dataset:  510698988  :  1
neurons with > 10.0 % explainable variance for dataset:  507304910  :  11
neurons with > 10.0 % explainable variance for dataset:  505687912  :  43
neurons with > 10.0 % explainable variance for dataset:  510518066  :  33
neurons with > 10.0 % explainable variance for dataset:  506539927  :  9
neurons with > 10.0 % explainable variance for dataset:  505696248  :  24
neurons with > 10.0 % explainable variance for dataset:  506520696  :  10
neurons with > 10.0 % explainable variance for dataset:  502254330  :  25
neurons with > 10.0 % explainable variance for dataset:  503772253  :  21


In [5]:
for key in data_dict.keys():
    dxcm, dxtime = data_set[key].get_running_speed()
    
    data, stim_array, rs = data_dict[key]
    print key
    
    print np.isnan(dxcm)

505687912
[ True  True  True ...,  True  True  True]
503866276
[ True  True  True ...,  True  True  True]
504508104
[ True  True  True ...,  True  True  True]
509517577
[ True  True  True ...,  True  True  True]
510698988
[ True  True  True ...,  True  True  True]
507304910
[ True  True  True ...,  True  True  True]
512145745
[ True  True  True ...,  True  True  True]
510518066
[ True  True  True ...,  True  True  True]
506539927
[ True  True  True ...,  True  True  True]
505696248
[ True  True  True ...,  True  True  True]
506520696
[ True  True  True ...,  True  True  True]
502254330
[ True  True  True ...,  True  True  True]
503772253
[ True  True  True ...,  True  True  True]


In [29]:
data_set[key].get_running_speed()

(array([ nan,  nan,  nan, ...,  nan,  nan,  nan]),
 array([   17.23731,    17.27056,    17.30381, ...,  3802.9257 ,
         3802.95895,  3802.9922 ]))

In [6]:
ds = data_set[key]

In [11]:

tb_data = arrange_data_rs(data_set)


In [18]:
for key in tb_data.keys():
    print np.isnan(tb_data[key]['natural_rs'])
    break

[False False False ..., False False False]


In [19]:
tb_data[key]['natural_rs']

array([ 0.        ,  7.28360748,  6.41263533, ..., -4.78502798,
       -0.79814446,  3.08645272])